In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import preprocessing # Import preprocessing from experimental
from tensorflow.keras.layers import Normalization # Import Normalization from layers


import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.head()

In [ ]:
df = dataset
df["sex"] = pd.factorize(df["sex"])[0]
df["region"] = pd.factorize(df["region"])[0]
df["smoker"] = pd.factorize(df["smoker"])[0]
dataset = df
dataset.head()

In [ ]:
dftest = dataset.sample(frac=0.2)
len(dftest)


In [ ]:
dftrain = dataset[~dataset.isin(dftest)].dropna()
len(dftrain)

In [ ]:
dftrain.head()

In [ ]:
train_labels = dftrain.pop("expenses")
train_labels.head()

In [ ]:
test_labels = dftest.pop("expenses")
test_labels.head()

In [ ]:
normalizer = Normalization()
normalizer.adapt(np.array(dftrain))

model = keras.Sequential([
    normalizer,
    layers.Dense(16),
    layers.Dense(8),#neurons chosen to provide a good trade-off between performance and complexity.
    layers.Dense(4),
    #layers.Dropout(.2), #Not always needed, sometimes better without
    layers.Dense(1),
])
# Its primary function is to normalize the input data, specifically by performing feature scaling. This involves adjusting the values of each feature to a common scale(relative), typically by centering them around zero and scaling them to have unit variance.
# Complexity of the data: More complex datasets with intricate patterns often benefit from larger(more neurons) hidden layers to capture those relationships, also increases the risk of overfitting

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mae', # (Mean Absolute Error)loss function, which measures the difference between the model's predictions and the actual values
    metrics=['mae', 'mse']
)
model.build()
model.summary()

#This specifies the optimization algorithm used to update the model's weights during training. The Adam optimizer is a choice in deep learning, with a learning rate of 0.1. The learning_rate controls how big the adjustments to the model's weights are during each training step.

In [ ]:
history = model.fit(
    dftrain,
    train_labels,
    epochs=100,
    validation_split=0.5,
    verbose=1, # disable logging
)

print(history)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(dftest, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(dftest).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
